# Projet : Physique des matériaux
## Louis Noirot
###### LMAPR1492

In [1]:
import numpy as np
import itertools
from mp_api.client import MPRester

from pymatgen.core.operations import SymmOp
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
from pymatgen.electronic_structure.plotter import BSPlotter
from pymatgen.phonon.plotter import PhononBSPlotter
from pymatgen.analysis.structure_matcher import StructureMatcher
from pymatgen.io.cif import CifWriter
from pymatgen.electronic_structure.core import Spin
from pymatgen.analysis.diffraction.xrd import XRDCalculator
from pymatgen.symmetry.bandstructure import HighSymmKpath

from plotly.subplots import make_subplots
import plotly.graph_objects as go

import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib.patches import Rectangle

from jupyter_jsmol.pymatgen import quick_view
from jupyter_jsmol import JsmolView
from ipywidgets import Layout, widgets, interact
from IPython.display import display

In [2]:
mp_key = "g2nCFD5rMkPRt9qdpOGhbfHJf2mgbv5x"
mp_id = "mp-14116"

In [3]:
hbarre = 6.62607015e-34/(2*np.pi)
eV = 1.6022e-19 # pour passer de J à eV
A = 1e-10
toHz = 1e12 # pour passer de THz à Hz

In [4]:
with MPRester(mp_key) as m:
    prim_struc = m.get_structure_by_material_id(mp_id)
    el_bs = m.get_bandstructure_by_material_id(mp_id)
    el_dos = m.get_dos_by_material_id(mp_id)
    ph_bs = m.get_phonon_bandstructure_by_material_id(mp_id)
    ph_dos = m.get_phonon_dos_by_material_id(mp_id)
    structure = m.get_structure_by_material_id(mp_id)
    band = m.get_bandstructure_by_material_id(mp_id)
conv_struc = SpacegroupAnalyzer(prim_struc).get_conventional_standard_structure()
symmops = SpacegroupAnalyzer(conv_struc).get_space_group_operations()

Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving ElectronicStructureDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving ElectronicStructureDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving PhononBSDOSDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving PhononBSDOSDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving ElectronicStructureDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

In [5]:
# Exporter en CIF avec symétries
writer = CifWriter(conv_struc, symprec=0.01)
writer.write_file("data/CuRhO2_fixed.cif")

# Tâche 5 :
#### Étude de l’effet de 3 éléments de symétrie différents (pas l’identité) chacun pour 1 atome différent.

## Symétries : nombre total, sélection de 3 et determination du type:

In [6]:
# Détermination du nombre total de symmétries et sélection de trois symétries:

# Obtenir les opérations de symétrie
symmops = SpacegroupAnalyzer(conv_struc).get_space_group_operations()

# Nombre de symétries disponibles
print(f"* Symétries disponibles : {len(symmops)}.")

indices = [5, 10, 25]
for idx in indices:
    op = symmops[idx]
    mat = op.rotation_matrix
    det = np.linalg.det(op.rotation_matrix)     # Car si det > 0 -> rotation ou identité
                                                #  et si det < 0 -> réflexion ou roto-inversion
        
    trace = np.trace(op.rotation_matrix)        # Car si trace = -3 -> inversion pure
                                                #  et si trace =  3 -> identité
        
    # Détermination du type de symétrie
    if np.allclose(mat, np.eye(3)):
        sym_type = "Identité"
        
    elif np.allclose(mat, -np.eye(3)):
        sym_type = "Inversion pure"
    
    elif det > 0:
        angle = np.arccos((trace-1)/2)*180/np.pi
        axis = np.array([
            mat[2,1]-mat[1,2],
            mat[0,2]-mat[2,0],
            mat[1,0]-mat[0,1]])
        axis = axis/np.linalg.norm(axis) if np.linalg.norm(axis) != 0 else "[indéfini]"
        sym_type = f"Rotation pure de {angle:.1f}° autour de l'axe {axis}"
    else:
        # Si la matrice au carré est égale à l'identité -> réflexion pure
        if np.allclose(np.dot(mat, mat), np.eye(3)):
            sym_type = "Réflexion pure"
        else:
            angle = np.arccos((trace+1)/2)*180/np.pi
            sym_type = f"Roto-inversion de {angle:.1f}°"

    # Affichage
    print(f"\n* Symétrie {idx} : {sym_type}")
    print("  Matrice de rotation:\n", mat)
    print("  Vecteur de translation :", op.translation_vector)

* Symétries disponibles : 36.

* Symétrie 5 : Roto-inversion de 60.0°
  Matrice de rotation:
 [[ 1. -1.  0.]
 [ 1.  0.  0.]
 [ 0.  0. -1.]]
  Vecteur de translation : [0. 0. 0.]

* Symétrie 10 : Rotation pure de 180.0° autour de l'axe [ 0.  0. -1.]
  Matrice de rotation:
 [[-1.  0.  0.]
 [-1.  1.  0.]
 [ 0.  0. -1.]]
  Vecteur de translation : [0. 0. 0.]

* Symétrie 25 : Inversion pure
  Matrice de rotation:
 [[-1.  0.  0.]
 [ 0. -1.  0.]
 [ 0.  0. -1.]]
  Vecteur de translation : [0.33333333 0.66666667 0.66666667]


## Sélection de trois éléments différents:

In [7]:
# Permet de voir la position des différents atomes dans la structure
print("* Voici la position des différens atomes dans la structure:")
for i, site in enumerate(conv_struc):
    print(f"   Atome {i} : {site.species_string}, position {site.frac_coords}")
    
# Sélection de 3 atomes spécifiques
atoms = [1, 4, 10]  # Indices des atomes sélectionés
atom_name = ["Cu", "Rh", "O"]

# Sélectionner 3 opérations de symétrie différentes
sym_op1 = symmops[5]
sym_op2 = symmops[10]
sym_op3 = symmops[25]

# Positions initiales
positions = [
    conv_struc[atoms[0]].frac_coords,
    conv_struc[atoms[1]].frac_coords,
    conv_struc[atoms[2]].frac_coords]

# Aplication des transformations aux atomes sélectionnés
transformed_positions = [
    sym_op1.operate(conv_struc[atoms[0]].frac_coords),
    sym_op2.operate(conv_struc[atoms[1]].frac_coords),
    sym_op3.operate(conv_struc[atoms[2]].frac_coords)]

# Afficher les anciennes et nouvelles positions des atomes transformés
print("\n\n * Position originale et transformée des différents atomes sélectionnés:")
for i in range(len(positions)):
    print(f"   Atome {atom_name[i]}: {positions[i]} -> {transformed_positions[i]}")

* Voici la position des différens atomes dans la structure:
   Atome 0 : Cu, position [0. 0. 0.]
   Atome 1 : Cu, position [0.66666667 0.33333333 0.33333333]
   Atome 2 : Cu, position [0.33333333 0.66666667 0.66666667]
   Atome 3 : Rh, position [0.33333333 0.66666667 0.16666667]
   Atome 4 : Rh, position [1.  1.  0.5]
   Atome 5 : Rh, position [0.66666667 0.33333333 0.83333333]
   Atome 6 : O, position [0.66666667 0.33333333 0.22617433]
   Atome 7 : O, position [0.       0.       0.107159]
   Atome 8 : O, position [0.33333333 0.66666667 0.55950767]
   Atome 9 : O, position [0.66666667 0.33333333 0.44049233]
   Atome 10 : O, position [0.       0.       0.892841]
   Atome 11 : O, position [0.33333333 0.66666667 0.77382567]


 * Position originale et transformée des différents atomes sélectionnés:
   Atome Cu: [0.66666667 0.33333333 0.33333333] -> [ 0.33333333  0.66666667 -0.33333333]
   Atome Rh: [1.  1.  0.5] -> [-1.00000000e+00 -1.11022302e-16 -5.00000000e-01]
   Atome O: [0.       0. 

## 0. La structure:

In [12]:
view = JsmolView(layout=Layout(height="600px"))
display(view)
view.load_file('data/CuRhO2_fixed.cif', '{1 1 1}', inline=True)
# view from right
view.script('moveto 0 0 1 0 90 200;')
view.script('rotate on')

JsmolView(layout=Layout(height='600px'))

## 1. La roto-inversion de 60° en fonction du Cu:

In [16]:
view1 = JsmolView(layout=Layout(height="600px"))
display(view1)
view1.load_file('data/CuRhO2_fixed.cif', '{1 1 1}', inline=True)
view1.script('draw SYMOP 5 {atomno=6}; select atomno=6; color yellow; label on')
view1.script('moveto 0.6 0.6 1 -0.1 210 300;')

JsmolView(layout=Layout(height='600px'))

## 2. La rotation pure de 180.0° autour de l'axe [0. 0. -1.] en fonction du Rh:

In [18]:
view2 = JsmolView(layout=Layout(height="600px"))
display(view2)
view2.load_file('data/CuRhO2_fixed.cif', '{1 1 1}', inline=True)
view2.script('draw SYMOP 10 {atomno=2}; select atomno=2; color yellow; label on')
view2.script('moveto 0.6 0.6 0.6 0 30 250;')

JsmolView(layout=Layout(height='600px'))

## 3. L'inversion pure avec un vecteur de translation non nul en function de l'O:

In [19]:
view3 = JsmolView(layout=Layout(height="600px"))
display(view3)
view3.load_file('data/CuRhO2_fixed.cif', '{1 1 1}', inline=True)
view3.script('draw SYMOP 25 {atomno=5}; select atomno=5; color yellow; label on')
view3.script('moveto 0.6 0.6 -0.6 0 -45 300;')

JsmolView(layout=Layout(height='600px'))